In [ ]:
import pandas as pd
import os
from dataRetrieval import *

### Data transformation : from excel to dateframe

In [ ]:
path = 'data/xls/'

table1 = read_from_excel_file('jester-data-1.xls', path)
table2 = read_from_excel_file('jester-data-2.xls', path)
table3 = read_from_excel_file('jester-data-3.xls', path)

print 'Shape of table1 : {}'.format(table1.shape)
print 'Shape of table2 : {}'.format(table2.shape)
print 'Shape of table3 : {}'.format(table3.shape)

frames12 = [table1, table2]
df12 = pd.concat(frames12)
print 'Shape of result : {}'.format(df12.shape)

frames13 = [table1, table3]
df13 = pd.concat(frames13)
print 'Shape of result : {}'.format(df13.shape)

frames23 = [table2, table3]
df23 = pd.concat(frames23)
print 'Shape of result : {}'.format(df23.shape)

store_to_csv(df, 'data/csv/', 'jester-12-data.csv')
store_to_csv(df, 'data/csv/', 'jester-13-data.csv')
store_to_csv(df, 'data/csv/', 'jester-23-data.csv')
store_to_csv(table1, 'data/csv/', 'jester-1-data.csv')
store_to_csv(table2, 'data/csv/', 'jester-2-data.csv')
store_to_csv(table3, 'data/csv/', 'jester-3-data.csv')

### Data loading

In [5]:
from dataRetrieval import *
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import log_loss
from pyfm import pylibfm
from time import time

#### Need about ~5 sec

In [2]:
t0 = time()
data12 = read_from_csv_file('jester-12-data.csv')
data13 = read_from_csv_file('jester-13-data.csv')
data23 = read_from_csv_file('jester-23-data.csv')
data1 = read_from_csv_file('jester-1-data.csv')
data2 = read_from_csv_file('jester-2-data.csv')
data3 = read_from_csv_file('jester-3-data.csv')

df12 = data12.drop(['#rating'], axis=1).copy()
df13 = data13.drop(['#rating'], axis=1).copy()
df23 = data23.drop(['#rating'], axis=1).copy()
df1 = data1.drop(['#rating'], axis=1).copy()
df2 = data2.drop(['#rating'], axis=1).copy()
df3 = data3.drop(['#rating'], axis=1).copy()
print 'done in %0.3fs.' % (time() - t0)

done in 4.913s.


#### Need about ~30 sec

In [3]:
t0 = time()
(train_data, y_train, train_users, train_items) = load_data(df12)
(test_data, y_test, test_users, test_items) = load_data(df3)
print 'done in %0.3fs.' % (time() - t0)

done in 29.767s.


#### Need about ~30 sec

In [4]:
t0 = time()
v = DictVectorizer()
X_train = v.fit_transform(train_data)
X_test = v.transform(test_data)
print 'done in %0.3fs.' % (time() - t0)

done in 30.696s.


#### Need about ~16 min for 10 iterations

In [6]:
t0 = time()
fm = pylibfm.FM(num_factors=10, num_iter=10, verbose=True, task="regression", initial_learning_rate=0.001, learning_rate_schedule="optimal")
fm.fit(X_train,y_train)
print 'done in %0.3fs.' % (time() - t0)

Creating validation dataset of 0.01 of training for adaptive regularization
-- Epoch 1
Training MSE: 11.41554
-- Epoch 2
Training MSE: 9.09404
-- Epoch 3
Training MSE: 8.21065
-- Epoch 4
Training MSE: 7.66758
-- Epoch 5
Training MSE: 7.30109
-- Epoch 6
Training MSE: 7.09915
-- Epoch 7
Training MSE: 6.98779
-- Epoch 8
Training MSE: 6.92607
-- Epoch 9
Training MSE: 6.88275
-- Epoch 10
Training MSE: 6.85400
done in 968.310s.


In [35]:
pred = np.round(fm.predict(X_test))
y_test = np.round(y_test)
print "Validation log loss: %.4f" % log_loss(y_test, pred)

ValueError: y_true and y_pred have different number of classes 21, 2

In [38]:
print np.unique(y_test)
print np.unique(pred)

[-10.  -9.  -8.  -7.  -6.  -5.  -4.  -3.  -2.  -1.  -0.   1.   2.   3.   4.
   5.   6.   7.   8.   9.  10.]
[-4. -3. -2. -1.  0.  1.  2.  3.]
